# Imports

In [1]:
import pandas as pd
import os
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
from scipy.stats import mode
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.metrics import r2_score
plt.style.use('dark_background')

from keras.layers import BatchNormalization, Dense, Input, Dropout
from keras.models import Model
from keras import backend as K
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping, ModelCheckpoint

2022-06-03 12:15:28.079799: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-03 12:15:28.079960: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Loading the Dataframe

In [9]:
pwd

'/home/rafael/code/rafabertolace/OnThePitch/notebooks'

## Merging the Seasons csv files (2019-2020 untill 2021-2022)

In [10]:
print(os.listdir('../raw_data/'))

['france', 'germany', 'eredivisie', 'turkey', 'england', 'greece', 'italy', 'spain', 'scotland', 'portugal', 'belgium']


In [11]:
def get_data_5(league1, league2=None, league3=None, league4=None, League5=None):
    files = [file for file in listdir(f'./../raw_data/{league1}')]
    data = pd.DataFrame()
    #For the case we have 2 leagues to concatenate:
    if league2:
        files1 = [file for file in listdir(f'./../raw_data/{league1}')]
        files2 = [file for file in listdir(f'./../raw_data/{league2}')]
        for file1 in files1:
            df = pd.read_csv(f'./../raw_data/{league1}/'+file1)
            data = pd.concat([data, df])
        for file2 in files2:
            df = pd.read_csv(f'./../raw_data/{league2}/'+file2)
            data = pd.concat([data, df])
        return data
    #For the case we have 3 leagues to concatenate:
    if league2 and league3:
        files1 = [file for file in listdir(f'./../raw_data/{league1}')]
        files2 = [file for file in listdir(f'./../raw_data/{league2}')]
        files3 = [file for file in listdir(f'./../raw_data/{league3}')]
        for file1 in files1:
            df = pd.read_csv(f'./../raw_data/{league1}/'+file1)
            data = pd.concat([data, df])
        for file2 in files2:
            df = pd.read_csv(f'./../raw_data/{league2}/'+file2)
            data = pd.concat([data, df])
        for file3 in files3:
            df = pd.read_csv(f'./../raw_data/{league3}/'+file3)
            data = pd.concat([data, df])
        return data
    #For the case we have 4 leagues to concatenate:
    if league2 and league3 and league4:
        files1 = [file for file in listdir(f'./../raw_data/{league1}')]
        files2 = [file for file in listdir(f'./../raw_data/{league2}')]
        files3 = [file for file in listdir(f'./../raw_data/{league3}')]
        files4 = [file for file in listdir(f'./../raw_data/{league4}')]
        for file1 in files1:
            df = pd.read_csv(f'./../raw_data/{league1}/'+file1)
            data = pd.concat([data, df])
        for file2 in files2:
            df = pd.read_csv(f'./../raw_data/{league2}/'+file2)
            data = pd.concat([data, df])
        for file3 in files3:
            df = pd.read_csv(f'./../raw_data/{league3}/'+file3)
            data = pd.concat([data, df])
        for file4 in files4:
            df = pd.read_csv(f'./../raw_data/{league4}/'+file4)
            data = pd.concat([data, df])
        return data
    #For the case we have 5 leagues to concatenate:
    if league2 and league3 and league4 and league5:
        files1 = [file for file in listdir(f'./../raw_data/{league1}')]
        files2 = [file for file in listdir(f'./../raw_data/{league2}')]
        files3 = [file for file in listdir(f'./../raw_data/{league3}')]
        files4 = [file for file in listdir(f'./../raw_data/{league4}')]
        files5 = [file for file in listdir(f'./../raw_data/{league5}')]
        for file1 in files1:
            df = pd.read_csv(f'./../raw_data/{league1}/'+file1)
            data = pd.concat([data, df])
        for file2 in files2:
            df = pd.read_csv(f'./../raw_data/{league2}/'+file2)
            data = pd.concat([data, df])
        for file3 in files3:
            df = pd.read_csv(f'./../raw_data/{league3}/'+file3)
            data = pd.concat([data, df])
        for file4 in files4:
            df = pd.read_csv(f'./../raw_data/{league4}/'+file4)
            data = pd.concat([data, df])
        for file5 in files5:
            df = pd.read_csv(f'./../raw_data/{league5}/'+file5)
            data = pd.concat([data, df])
        return data

In [12]:
def get_data(league1, wall=False):
    if wall:
        data = pd.DataFrame()
        leagues = os.listdir(f'./../raw_data/')
        data = pd.DataFrame()
        for league in leagues:
            files = os.listdir(f'./../raw_data/{league}')
            for file in files:
                df = pd.read_csv((f'./../raw_data/{league}/'+file))
                data = pd.concat([data, df])
        return data
    else:
        files = [file for file in os.listdir(f'./../raw_data/{league1}')]
        data = pd.DataFrame()
        for file in files:
            df = pd.read_csv(f'./../raw_data/{league1}/'+file)
            data = pd.concat([data, df])
        return data

In [13]:
# data = get_data_5('england', league2='eredivisie', league3='portugal', league4='scotland', League5='spain')

In [21]:
data = get_data('england')

In [22]:
data.head()

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,E0,12/09/2020,12:30,Fulham,Arsenal,0,3,A,0,1,...,1.84,0.75,2.01,1.89,2.02,1.91,2.13,1.92,2.02,1.87
1,E0,12/09/2020,15:00,Crystal Palace,Southampton,1,0,H,1,0,...,1.70,0.25,1.78,2.13,1.79,2.17,1.85,2.18,1.79,2.12
2,E0,12/09/2020,17:30,Liverpool,Leeds,4,3,H,3,2,...,2.62,-1.50,1.85,2.05,1.85,2.08,1.90,2.16,1.84,2.04
3,E0,12/09/2020,20:00,West Ham,Newcastle,0,2,A,0,0,...,1.92,-0.50,2.03,1.87,2.04,1.88,2.09,1.91,2.02,1.86
4,E0,13/09/2020,14:00,West Brom,Leicester,0,3,A,0,0,...,1.73,0.25,1.92,1.98,1.93,1.99,1.95,2.01,1.91,1.97


In [23]:
for col_name in data.columns: 
    print(col_name)

Div
Date
Time
HomeTeam
AwayTeam
FTHG
FTAG
FTR
HTHG
HTAG
HTR
Referee
HS
AS
HST
AST
HF
AF
HC
AC
HY
AY
HR
AR
B365H
B365D
B365A
BWH
BWD
BWA
IWH
IWD
IWA
PSH
PSD
PSA
WHH
WHD
WHA
VCH
VCD
VCA
MaxH
MaxD
MaxA
AvgH
AvgD
AvgA
B365>2.5
B365<2.5
P>2.5
P<2.5
Max>2.5
Max<2.5
Avg>2.5
Avg<2.5
AHh
B365AHH
B365AHA
PAHH
PAHA
MaxAHH
MaxAHA
AvgAHH
AvgAHA
B365CH
B365CD
B365CA
BWCH
BWCD
BWCA
IWCH
IWCD
IWCA
PSCH
PSCD
PSCA
WHCH
WHCD
WHCA
VCCH
VCCD
VCCA
MaxCH
MaxCD
MaxCA
AvgCH
AvgCD
AvgCA
B365C>2.5
B365C<2.5
PC>2.5
PC<2.5
MaxC>2.5
MaxC<2.5
AvgC>2.5
AvgC<2.5
AHCh
B365CAHH
B365CAHA
PCAHH
PCAHA
MaxCAHH
MaxCAHA
AvgCAHH
AvgCAHA


# Over/Under DF

In [24]:
data.isnull().sum().sort_values(ascending=False)

IWCA        3
IWCD        3
IWCH        3
B365<2.5    1
B365>2.5    1
           ..
IWD         0
IWH         0
BWA         0
BWD         0
AvgCAHA     0
Length: 106, dtype: int64

In [25]:
# data.dropna(inplace=True)

In [26]:
data

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,E0,12/09/2020,12:30,Fulham,Arsenal,0,3,A,0,1,...,1.84,0.75,2.01,1.89,2.02,1.91,2.13,1.92,2.02,1.87
1,E0,12/09/2020,15:00,Crystal Palace,Southampton,1,0,H,1,0,...,1.70,0.25,1.78,2.13,1.79,2.17,1.85,2.18,1.79,2.12
2,E0,12/09/2020,17:30,Liverpool,Leeds,4,3,H,3,2,...,2.62,-1.50,1.85,2.05,1.85,2.08,1.90,2.16,1.84,2.04
3,E0,12/09/2020,20:00,West Ham,Newcastle,0,2,A,0,0,...,1.92,-0.50,2.03,1.87,2.04,1.88,2.09,1.91,2.02,1.86
4,E0,13/09/2020,14:00,West Brom,Leicester,0,3,A,0,0,...,1.73,0.25,1.92,1.98,1.93,1.99,1.95,2.01,1.91,1.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,E0,22/05/2022,16:00,Crystal Palace,Man United,1,0,H,1,0,...,2.04,0.25,1.68,2.15,1.74,2.23,1.88,2.25,1.74,2.16
376,E0,22/05/2022,16:00,Leicester,Southampton,4,1,H,0,0,...,2.63,-0.75,1.83,2.07,1.88,2.03,1.94,2.26,1.87,2.01
377,E0,22/05/2022,16:00,Liverpool,Wolves,3,1,H,1,1,...,3.28,-2.50,2.02,1.77,2.06,1.83,2.19,1.99,2.07,1.80
378,E0,22/05/2022,16:00,Man City,Aston Villa,3,2,H,0,1,...,3.36,-2.25,2.06,1.84,2.05,1.86,2.09,2.03,2.01,1.87


# Features Engineering

### Number of Goals, Over and Under

In [27]:
# total number of goals = goals from the home team + goals from visiting team
data['nb_goals']=data['FTHG']+data['FTAG']

# boolean: true or false regarding whether they were more than 2.5 goals
data['over_2.5_goals']=data['nb_goals']>2.5

# boolean: true or false regarding whether they were less than 2.5 goals
data['under_2.5_goals']=data['nb_goals']<2.5

### Payout

  Payout of betting on over/under 2.5 goals: we get 0 if we lose the bet, we get the Avg if we win the bet (Avg = market average of the odds)


#### Payout Opening

In [28]:
# payout under 2.5 for Average OPENING odds
data['Payout Under AVG'] = data['under_2.5_goals']*data['Avg<2.5']

# payout over 2.5 for Average OPENING odds
data['Payout Over AVG'] = data['over_2.5_goals']*data['Avg>2.5']

#payout UNDER 2.5 for PINACLE specifically
data['Payout Under P'] = data['under_2.5_goals']*data['P<2.5']

#payout OVER 2.5 for PINACLE specifically
data['Payout Over P'] = data['over_2.5_goals']*data['P>2.5']

#payout UNDER 2.5 for 365 specifically
data['Payout Under B365'] = data['under_2.5_goals']*data['B365<2.5']

#payout OVER 2.5 for 365 specifically
data['Payout Over B365'] = data['over_2.5_goals']*data['B365>2.5']

#### Payout Closing

In [29]:
# payout under 2.5 for Average CLOSING odds
data['Payout Under AVGC'] = data['under_2.5_goals']*data['AvgC<2.5']

# payout over 2.5 for Average CLOSING odds
data['Payout Over AVG'] = data['over_2.5_goals']*data['AvgC>2.5']

#payout UNDER 2.5 for PINACLE closing ddds specifically
data['Payout Under PC'] = data['under_2.5_goals']*data['PC<2.5']

#payout OVER 2.5 for PINACLE closing odds specifically
data['Payout Over PC'] = data['over_2.5_goals']*data['PC>2.5']

#payout UNDER 2.5 for 365 closing odds specifically
data['Payout Under B365C'] = data['under_2.5_goals']*data['B365C<2.5']

#payout OVER 2.5 for 365 closing odds specifically
data['Payout Over B365C'] = data['over_2.5_goals']*data['B365C>2.5']

### Implied Probability

#### Implied Probability Opening

In [30]:
#Implied Probability UNDER 2.5 goals for for overall market opening odds (Avg) 
data['IP Under AVG']=1/data['Avg<2.5']*100

#Implied Probability OVER 2.5 goals for for overall market opening odds (Avg) 
data['IP Over AVG']=1/data['Avg>2.5']*100

#Implied Probability UNDER 2.5 goals for PINACLE
data['IP Under P']=1/data['P<2.5']*100

#Implied Probability OVER 2.5 goals for PINACLE
data['IP Over P']=1/data['P>2.5']*100

#Implied Probability UNDER 2.5 goals for 365
data['IP Under B365']=1/data['B365<2.5']*100

#Implied Probability OVER 2.5 goals for 365
data['IP Over B365']=1/data['B365>2.5']*100

#### Implied Probability Closing

In [31]:
#Implied Probability UNDER 2.5 goals for overall market closing odds (AvgC)
data['IP Under AVGC']=1/data['AvgC<2.5']*100

#Implied Probability OVER 2.5 goals for overall market closing odds (AvgC)
data['IP Over AVGC']=1/data['AvgC>2.5']*100
 
#Implied Probability UNDER 2.5 goals for PINACLE closing odds
data['IP Under PC']=1/data['PC<2.5']*100

#Implied Probability OVER 2.5 goals for PINACLE closing odds
data['IP Over PC']=1/data['PC>2.5']*100

#Implied Probability UNDER 2.5 goals for 365 closing odds
data['IP Under B365C']=1/data['B365C<2.5']*100

#Implied Probability OVER 2.5 goals for 365 closing odds
data['IP Over B365C']=1/data['B365C>2.5']*100

### Binning the implied probabilities

#### Binning IP Opening

In [32]:
bins = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100]

#Binning UNDER 2.5 Average Market opening odds
data['Bin IP Under AVG'] = pd.cut(data['IP Under AVG'], bins)

#Binning Over 2.5 Average Market opening odds
data['Bin IP Over AVG'] = pd.cut(data['IP Over AVG'], bins)

#Binned UNDER 2.5 Pinnacle opening odds
data['Bin IP Under P'] = pd.cut(data['IP Under P'], bins)

#Binned OVER 2.5 Pinnacle
data['Bin IP Over P'] = pd.cut(data['IP Over P'], bins)

#Binned UNDER 2.5 bet365 OPENING odds
data['Bin IP Under B365'] = pd.cut(data['IP Under B365'], bins)

#Binned OVER 2.5 bet365 OPENING odds
data['Bin IP Over B365'] = pd.cut(data['IP Over B365'], bins)

#### Binning IP Closing

In [33]:
bins = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100]

#Binning UNDER 2.5 Average Market closing odds
data['Bin IP Under AVGC'] = pd.cut(data['IP Under AVGC'], bins)

#Binning OVER 2.5 Average Market closing odds
data['Bin IP Over AVGC'] = pd.cut(data['IP Over AVGC'], bins)

#Binned UNDER 2.5 Pinnacle closing odds
data['Bin IP Under PC'] = pd.cut(data['IP Under PC'], bins)

#Binned OVER 2.5 Pinnacle CLOSING odds
data['Bin IP Over PC'] = pd.cut(data['IP Over PC'], bins)

#Binned UNDER 2.5 bet365 CLOSING odds
data['Bin IP Under B365C'] = pd.cut(data['IP Under B365C'], bins)

#Binned OVER 2.5 bet365 CLOSING odds
data['Bin IP Over B365C'] = pd.cut(data['IP Over B365C'], bins)

### Binning the odds

#### Binning Odds Opening

In [34]:
bins = [1, 1.5, 2, 3, 99999]

#Binning UNDER 2.5 Average Market opening odds
data['Bin Odds Under AVG'] = pd.cut(data['Avg<2.5'], bins)

#Binning Over 2.5 Average Market opening odds
data['Bin Odds Over AVG'] = pd.cut(data['Avg>2.5'], bins)

#Binned UNDER 2.5 Pinnacle opening odds
data['Bin Odds Under P'] = pd.cut(data['P<2.5'], bins)

#Binned OVER 2.5 Pinnacle
data['Bin Odds Over P'] = pd.cut(data['P>2.5'], bins)

#Binned UNDER 2.5 bet365 OPENING odds
data['Bin Odds Under B365'] = pd.cut(data['B365<2.5'], bins)

#Binned OVER 2.5 bet365 OPENING odds
data['Bin Odds Over B365'] = pd.cut(data['B365>2.5'], bins)

#### Binning Odds Closing

In [35]:
bins = [1, 1.25, 2, 3, 99999]

#Binning UNDER 2.5 Average Market opening odds
data['Bin Odds Under AVGC'] = pd.cut(data['AvgC<2.5'], bins)

#Binning Over 2.5 Average Market opening odds
data['Bin Odds Over AVGC'] = pd.cut(data['AvgC>2.5'], bins)

#Binned UNDER 2.5 Pinnacle opening odds
data['Bin Odds Under PC'] = pd.cut(data['PC<2.5'], bins)

#Binned OVER 2.5 Pinnacle
data['Bin Odds Over PC'] = pd.cut(data['PC>2.5'], bins)

#Binned UNDER 2.5 bet365 OPENING odds
data['Bin Odds Under B365C'] = pd.cut(data['B365C<2.5'], bins)

#Binned OVER 2.5 bet365 OPENING odds
data['Bin Odds Over B365C'] = pd.cut(data['B365C>2.5'], bins)

### Other Features from D3

In [36]:
data['PC Pays Better AVGC Bool'] = data['PC<2.5'] > data['AvgC<2.5']

In [37]:
data['PC Pays Better AVGC Diff'] = data['PC<2.5'] / data['AvgC<2.5']

In [38]:
data['%vig_p'] = (1 - (1 / (1/data['PC>2.5'] + 1/data['PC<2.5'])))*100

In [39]:
data['%vig_avg'] = (1 - (1 / (1/data['AvgC>2.5'] + 1/data['AvgC<2.5'])))*100

In [40]:
data['PC Under Odds Less P Bool'] = data['PC<2.5'] < data['P<2.5']

In [41]:
data['PC Under Odds Less P Diff'] = data['PC<2.5'] / data['P<2.5']

### Opening Markets

In [74]:
#Average Market UNDER opening odds
fdf_under_avg = data[['IP Under AVG','under_2.5_goals','Bin Under AVG','Payout Under AVG']]

#Average Market OVER opening odds
fdf_over_avg = data[['IP Over AVG','over_2.5_goals','Bin Over AVG','Payout Over AVG']]

#pinnacle UNDER opening odds
fdf_under_pinacle = data[['IP Under P','under_2.5_goals','Bin Under P','Payout Under P']]

#pinnacle OVER opening odds
fdf_over_pinacle = data[['IP Over P','over_2.5_goals','Bin Over P','Payout Over P']]

#bet365 UNDER opening odds
fdf_under_365 = data[['IP Under B365','under_2.5_goals','Bin Under B365','Payout Under B365']]

#bet365 OVER opening odds
fdf_over_365 = data[['IP Over B365','over_2.5_goals','Bin Over B365','Payout Over B365']]

KeyError: "['Bin Under AVG'] not in index"

### Closing Markets

In [75]:
#Average Market UNDER opening odds
fdf_under_avg_closing = data[['IP Under AVGC','under_2.5_goals','Bin Under AVGC','Payout Under AVGC']]

#Average Market OVER opening odds
fdf_over_avg_closing = data[['IP Over AVGC','over_2.5_goals','Bin Over AVGC','Payout Over AVGC']]

#pinnacle UNDER opening odds
fdf_under_pinacle_closing = data[['IP Under PC','under_2.5_goals','Bin Under PC','Payout Under PC']]

#pinnacle OVER opening odds
fdf_over_pinacle_closing = data[['IP Over PC','over_2.5_goals','Bin Over PC','Payout Over PC']]

#bet365 UNDER opening odds
fdf_under_365_closing = data[['IP Under B365C','under_2.5_goals','Bin Under B365C','Payout Under B365C']]

#bet365 OVER opening odds
fdf_over_365_closing = data[['IP Over B365C','over_2.5_goals','Bin Over B365C','Payout Over B365C']]

KeyError: "['Bin Under AVGC'] not in index"

### Under

#### Opening Odds

In [27]:
fdf_under_avg_agg = fdf_under_avg.groupby('binned <2.5 avg')['payout_avg_under_2.5'].agg(['count', 'mean', 'median'])

fdf_under_pinacle_agg = fdf_under_pinacle.groupby('binned <2.5 pinacle')['payout_under_2.5_pinacle'].agg(['count', 'mean', 'median'])

fdf_under_bet365_agg = fdf_under_365.groupby('binned <2.5 365')['payout_under_2.5_365'].agg(['count', 'mean', 'median'])

In [28]:
fdf_under_avg_agg.rename(columns={"count": "count_avg", "mean": "mean_avg", "median": "median_avg"}, inplace=True)
fdf_under_avg_agg.index.names = ['bin<2.5']

fdf_under_pinacle_agg.rename(columns={"count": "count_p", "mean": "mean_p", "median": "median_p"}, inplace=True)
fdf_under_pinacle_agg.index.names = ['bin<2.5']

fdf_under_bet365_agg.rename(columns={"count": "count_b365", "mean": "mean_b365", "median": "median_b365"}, inplace=True)
fdf_under_bet365_agg.index.names = ['bin<2.5']

opening_test_1 = fdf_under_avg_agg.merge(fdf_under_pinacle_agg, on='bin<2.5')
opening_test_2 = opening_test_1.merge(fdf_under_bet365_agg, on='bin<2.5')
opening_test_2

,count_avg,mean_avg,median_avg,count_p,mean_p,median_p,count_b365,mean_b365,median_b365
bin<2.5,,,,,,,,,
"(0, 5]",0,NaN,NaN,0,NaN,NaN,0,NaN,NaN
"(5, 10]",0,NaN,NaN,0,NaN,NaN,0,NaN,NaN
"(10, 15]",0,NaN,NaN,0,NaN,NaN,0,NaN,NaN
"(15, 20]",0,NaN,NaN,0,NaN,NaN,0,NaN,NaN
"(20, 25]",1,0.000000,0.00,1,0.000000,0.000,1,0.000000,0.00
"(25, 30]",1,0.000000,0.00,3,0.000000,0.000,5,0.000000,0.00
"(30, 35]",10,0.295000,0.00,12,0.260000,0.000,9,0.344444,0.00
"(35, 40]",34,0.828529,0.00,47,0.997660,0.000,24,0.854167,0.00
"(40, 45]",152,0.760987,0.00,186,0.837742,0.000,134,0.728358,0.00


#### Closing Odds

In [29]:
fdf_under_avg_closing_gb_agg = fdf_under_avg_closing.groupby('binned <2.5 avg closing')['payout_avg_under_closing_2.5'].agg(['count', 'mean', 'median', mode])

fdf_under_pinacle_closing_gb_agg = fdf_under_pinacle_closing.groupby('binned <2.5 pinacle closing')['payout_under_2.5_pinacle_closing'].agg(['count', 'mean', 'median', mode])

fdf_under_bet365_closing_gb_agg = fdf_under_365_closing.groupby('binned <2.5 365 closing')['payout_under_2.5_365_closing'].agg(['count', 'mean', 'median', mode])

In [30]:
fdf_under_avg_closing_gb_agg.rename(columns={"count": "count_avg", "mean": "mean_avg", "median": "median_avg", "mode": "mode_avg"}, inplace=True)
fdf_under_avg_closing_gb_agg.index.names = ['bin<2.5']

fdf_under_pinacle_closing_gb_agg.rename(columns={"count": "count_p", "mean": "mean_p", "median": "median_p", "mode": "mode_p"}, inplace=True)
fdf_under_pinacle_closing_gb_agg.index.names = ['bin<2.5']

fdf_under_bet365_closing_gb_agg.rename(columns={"count": "count_b365", "mean": "mean_b365", "median": "median_b365", "mode": "mode_b365"}, inplace=True)
fdf_under_bet365_closing_gb_agg.index.names = ['bin<2.5']

closing_test_1 = fdf_under_avg_closing_gb_agg.merge(fdf_under_pinacle_closing_gb_agg, on='bin<2.5')
closing_test_2 = closing_test_1.merge(fdf_under_bet365_closing_gb_agg, on='bin<2.5')
closing_test_2['bins'] = closing_test_2.index
closing_test_2

,count_avg,mean_avg,median_avg,mode_avg,count_p,mean_p,median_p,mode_p,count_b365,mean_b365,median_b365,mode_b365,bins
bin<2.5,,,,,,,,,,,,,
"(0, 5]",0,NaN,NaN,"([], [])",0,NaN,NaN,"([], [])",0,NaN,NaN,"([], [])","(0, 5]"
"(5, 10]",0,NaN,NaN,"([], [])",0,NaN,NaN,"([], [])",0,NaN,NaN,"([], [])","(5, 10]"
"(10, 15]",0,NaN,NaN,"([], [])",0,NaN,NaN,"([], [])",0,NaN,NaN,"([], [])","(10, 15]"
"(15, 20]",0,NaN,NaN,"([], [])",0,NaN,NaN,"([], [])",0,NaN,NaN,"([], [])","(15, 20]"
"(20, 25]",1,0.000000,0.00,"([0.0], [1])",2,0.000000,0.00,"([0.0], [2])",2,0.000000,0.00,"([0.0], [2])","(20, 25]"
"(25, 30]",2,0.000000,0.00,"([0.0], [2])",6,0.566667,0.00,"([0.0], [5])",7,0.500000,0.00,"([0.0], [6])","(25, 30]"
"(30, 35]",14,0.233571,0.00,"([0.0], [13])",16,1.102500,0.00,"([0.0], [10])",15,0.956667,0.00,"([0.0], [10])","(30, 35]"
"(35, 40]",53,1.002642,0.00,"([0.0], [33])",76,0.559474,0.00,"([0.0], [60])",46,0.734783,0.00,"([0.0], [33])","(35, 40]"
"(40, 45]",162,0.741852,0.00,"([0.0], [110])",180,0.964500,0.00,"([0.0], [106])",143,0.779371,0.00,"([0.0], [95])","(40, 45]"


### Over

#### Opening Odds

In [31]:
fdf_over_avg_agg = fdf_over_avg.groupby('binned >2.5 avg')['payout_avg_over_2.5'].agg(['count', 'mean', 'median'])

fdf_over_pinacle_agg = fdf_over_pinacle.groupby('binned >2.5 pinacle')['payout_over_2.5_pinacle'].agg(['count', 'mean', 'median'])

fdf_over_bet365_agg = fdf_over_365.groupby('binned >2.5 365')['payout_over_2.5_365'].agg(['count', 'mean', 'median'])

In [32]:
fdf_over_avg_agg.rename(columns={"count": "count_avg", "mean": "mean_avg", "median": "median_avg"}, inplace=True)
fdf_over_avg_agg.index.names = ['bin>2.5']

fdf_over_pinacle_agg.rename(columns={"count": "count_p", "mean": "mean_p", "median": "median_p"}, inplace=True)
fdf_over_pinacle_agg.index.names = ['bin>2.5']

fdf_over_bet365_agg.rename(columns={"count": "count_b365", "mean": "mean_b365", "median": "median_b365"}, inplace=True)
fdf_over_bet365_agg.index.names = ['bin>2.5']

over_opening_test_1 = fdf_over_avg_agg.merge(fdf_over_pinacle_agg, on='bin>2.5')
over_opening_test_2 = over_opening_test_1.merge(fdf_over_bet365_agg, on='bin>2.5')
over_opening_test_2

,count_avg,mean_avg,median_avg,count_p,mean_p,median_p,count_b365,mean_b365,median_b365
bin>2.5,,,,,,,,,
"(0, 5]",0,NaN,NaN,0,NaN,NaN,0,NaN,NaN
"(5, 10]",0,NaN,NaN,0,NaN,NaN,0,NaN,NaN
"(10, 15]",0,NaN,NaN,0,NaN,NaN,0,NaN,NaN
"(15, 20]",0,NaN,NaN,0,NaN,NaN,0,NaN,NaN
"(20, 25]",0,NaN,NaN,0,NaN,NaN,0,NaN,NaN
"(25, 30]",0,NaN,NaN,0,NaN,NaN,0,NaN,NaN
"(30, 35]",0,NaN,NaN,0,NaN,NaN,0,NaN,NaN
"(35, 40]",2,1.310000,1.310,2,1.390000,1.39,2,1.300000,1.30
"(40, 45]",23,1.104348,0.000,46,0.912391,0.00,23,1.097826,0.00


#### Closing Odds

In [33]:
fdf_over_avg_closing_agg = fdf_over_avg_closing.groupby('binned >2.5 avg closing')['payout_avg_over_closing_2.5'].agg(['count', 'mean', 'median'])

fdf_over_pinacle_closing_agg = fdf_over_pinacle_closing.groupby('binned >2.5 pinacle closing')['payout_over_2.5_pinacle_closing'].agg(['count', 'mean', 'median'])

fdf_over_bet365_closing_agg = fdf_over_365_closing.groupby('binned >2.5 365 closing')['payout_over_2.5_365_closing'].agg(['count', 'mean', 'median'])

In [34]:
fdf_over_avg_closing_agg.rename(columns={"count": "count_avg", "mean": "mean_avg", "median": "median_avg"}, inplace=True)
fdf_over_avg_closing_agg.index.names = ['bin>2.5']

fdf_over_pinacle_closing_agg.rename(columns={"count": "count_p", "mean": "mean_p", "median": "median_p"}, inplace=True)
fdf_over_pinacle_closing_agg.index.names = ['bin>2.5']

fdf_over_bet365_closing_agg.rename(columns={"count": "count_b365", "mean": "mean_b365", "median": "median_b365"}, inplace=True)
fdf_over_bet365_closing_agg.index.names = ['bin>2.5']

over_closing_test_1 = fdf_over_avg_closing_agg.merge(fdf_over_pinacle_closing_agg, on='bin>2.5')
over_closing_test_2 = over_closing_test_1.merge(fdf_over_bet365_closing_agg, on='bin>2.5')
over_closing_test_2

,count_avg,mean_avg,median_avg,count_p,mean_p,median_p,count_b365,mean_b365,median_b365
bin>2.5,,,,,,,,,
"(0, 5]",0,NaN,NaN,0,NaN,NaN,0,NaN,NaN
"(5, 10]",0,NaN,NaN,0,NaN,NaN,0,NaN,NaN
"(10, 15]",0,NaN,NaN,0,NaN,NaN,0,NaN,NaN
"(15, 20]",0,NaN,NaN,0,NaN,NaN,0,NaN,NaN
"(20, 25]",0,NaN,NaN,0,NaN,NaN,0,NaN,NaN
"(25, 30]",0,NaN,NaN,0,NaN,NaN,0,NaN,NaN
"(30, 35]",0,NaN,NaN,1,0.000000,0.000,0,NaN,NaN
"(35, 40]",5,0.502000,0.000,12,1.072500,0.000,6,0.833333,0.00
"(40, 45]",46,1.256739,2.230,82,1.155732,1.115,53,1.083962,0.00


# Starting Model - XGBoost

In [42]:
data.head()

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,Bin Odds Under PC,Bin Odds Over PC,Bin Odds Under B365C,Bin Odds Over B365C,PC Pays Better AVGC Bool,PC Pays Better AVGC Diff,%vig_p,%vig_avg,PC Under Odds Less P Bool,PC Under Odds Less P Diff
0,E0,12/09/2020,12:30,Fulham,Arsenal,0,3,A,0,1,...,"(1.25, 2.0]","(2.0, 3.0]","(1.25, 2.0]","(1.25, 2.0]",True,1.010870,2.255102,4.166667,True,0.873239
1,E0,12/09/2020,15:00,Crystal Palace,Southampton,1,0,H,1,0,...,"(1.25, 2.0]","(2.0, 3.0]","(1.25, 2.0]","(2.0, 3.0]",True,1.011765,2.331658,4.484536,False,1.023810
2,E0,12/09/2020,17:30,Liverpool,Leeds,4,3,H,3,2,...,"(2.0, 3.0]","(1.25, 2.0]","(2.0, 3.0]","(1.25, 2.0]",True,1.053435,2.398126,4.611650,False,1.061538
3,E0,12/09/2020,20:00,West Ham,Newcastle,0,2,A,0,0,...,"(1.25, 2.0]","(1.25, 2.0]","(1.25, 2.0]","(1.25, 2.0]",False,1.000000,2.040816,4.250653,False,1.005236
4,E0,13/09/2020,14:00,West Brom,Leicester,0,3,A,0,0,...,"(1.25, 2.0]","(2.0, 3.0]","(1.25, 2.0]","(2.0, 3.0]",True,1.005780,2.261965,4.136598,True,0.910995


In [196]:
for col_name in data_xgb_booleans: 
    print(col_name)

Div
Date
Time
HomeTeam
AwayTeam
FTHG
FTAG
FTR
HTHG
HTAG
HTR
Referee
HS
AS
HST
AST
HF
AF
HC
AC
HY
AY
HR
AR
B365H
B365D
B365A
BWH
BWD
BWA
IWH
IWD
IWA
PSH
PSD
PSA
WHH
WHD
WHA
VCH
VCD
VCA
MaxH
MaxD
MaxA
AvgH
AvgD
AvgA
B365>2.5
B365<2.5
P>2.5
P<2.5
Max>2.5
Max<2.5
Avg>2.5
Avg<2.5
AHh
B365AHH
B365AHA
PAHH
PAHA
MaxAHH
MaxAHA
AvgAHH
AvgAHA
B365CH
B365CD
B365CA
BWCH
BWCD
BWCA
IWCH
IWCD
IWCA
PSCH
PSCD
PSCA
WHCH
WHCD
WHCA
VCCH
VCCD
VCCA
MaxCH
MaxCD
MaxCA
AvgCH
AvgCD
AvgCA
B365C>2.5
B365C<2.5
PC>2.5
PC<2.5
MaxC>2.5
MaxC<2.5
AvgC>2.5
AvgC<2.5
AHCh
B365CAHH
B365CAHA
PCAHH
PCAHA
MaxCAHH
MaxCAHA
AvgCAHH
AvgCAHA
nb_goals
over_2.5_goals
under_2.5_goals
Payout Under AVG
Payout Over AVG
Payout Under P
Payout Over P
Payout Under B365
Payout Over B365
Payout Under AVGC
Payout Under PC
Payout Over PC
Payout Under B365C
Payout Over B365C
IP Under AVG
IP Over AVG
IP Under P
IP Over P
IP Under B365
IP Over B365
IP Under AVGC
IP Over AVGC
IP Under PC
IP Over PC
IP Under B365C
IP Over B365C
Bin IP Under AVG
Bin I

In [ ]:
def odds_loss(y_true, y_pred):
    """
    The function implements the custom loss function mentioned in info.pdf
    
    Inputs
    true : a vector of dimension batch_size, 7. A label encoded version of the output and the backp1_a and backp1_b
    pred : a vector of probabilities of dimension batch_size , 5.
    
    Returns 
    the loss value
    """
    win_home_team = y_true[:, 0:1]
    win_home_or_draw = y_true[:, 1:2]
    win_away = y_true[:, 2:3]
    win_away_or_draw = y_true[:, 3:4]
    draw = y_true[:, 4:5]
    no_bet = y_true[:, 5:6]
    odds_a = y_true[:, 6:7]
    odds_b = y_true[:, 7:8]
    gain_loss_vector = K.concatenate([win_home_team * (odds_a - 1) + (1 - win_home_team) * -1,
                                      win_home_or_draw * (1/(1 -1/odds_b) - 1) + (1 - win_home_or_draw) * -1,
                                      win_away * (odds_b - 1) + (1 - win_away) * -1,
                                      win_away_or_draw * (1/(1 -1/odds_a) - 1) + (1 - win_away_or_draw) * -1,
                                      draw * (1/(1 - 1/odds_a - 1/odds_b) - 1) + (1 - draw) * -1,
                                      K.zeros_like(odds_a)], axis=1)
    return -1 * K.mean(K.sum(gain_loss_vector * y_pred, axis=1))

In [ ]:
def odds_loss(y_true, y_pred):
    """
    The function implements the custom loss function mentioned in info.pdf
    
    Inputs
    true : a vector of dimension batch_size, 7. A label encoded version of the output and the backp1_a and backp1_b
    pred : a vector of probabilities of dimension batch_size , 5.
    
    Returns 
    the loss value
    """
    